# SQL agent

## Setup

### Imports

In [73]:
import db_connect 

from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain import agents
from langchain.prompts import PromptTemplate

from langchain.agents.agent_toolkits.sql import base
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

### LangSmith

In [74]:
from langsmith import Client
import os

os.environ["LANGCHAIN_PROJECT"] = "text2sql"
client = Client()

### Load models

In [75]:
llama3 = Ollama(model="llama3:8b", temperature=0)
llama3_embeddings = OllamaEmbeddings(model="llama3:8b", temperature=0)

llama3_inst = Ollama(model="llama3:instruct", temperature=0)
llama3_inst_q8 = Ollama(model="llama3:8b-instruct-q8_0", temperature=0)
llama3_inst_fp16 = Ollama(model="llama3:8b-instruct-fp16", temperature=0)

### Connect to DB with Readonly role

In [76]:
db = db_connect.get_db()

#### Check connection

In [77]:
db.run("select * from passenger")

"[(16, 'John'), (17, 'James'), (18, 'Poul'), (19, 'Christofer'), (20, 'Superman')]"

-----

In [78]:
template =\
"""# Personality
You are an expert in PostgreSQL. Answer the question of an ordinary employee who does not know SQL. Please reply in plain text.
# Task
Please answer the following question to the best of your ability.
# Tools
Use the following tools to answer:
{tools}
# Format
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action... 
(this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

IF YOU WROTE \"FINAL ANSWER\". THEN DON'T WRITE ANYTHING MORE
# Start chain of thoughts

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

def err_handle(err):
    return "# Error\n" +\
        f"The previous steps caused an error, here is its text: \n\"{err}\"\n" +\
        "DO NOT OUTPUT AN ACTION AND A FINAL ANSWER AT THE SAME TIME"

# agent = base.create_sql_agent(
#     llm=llama3,
#     db=db,
#     agent_type=agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     prompt=prompt,
#     verbose=True,
#     early_stopping_method="generate",
#     agent_executor_kwargs={
#         "handle_parsing_errors": err_handle,
#     }
# )

model = llama3_inst_fp16
tools = SQLDatabaseToolkit(db=db, llm=model).get_tools()

agent = agents.create_react_agent(
    llm=model,
    prompt=prompt,
    tools=tools,
)

agent = agents.AgentExecutor(
        name="SQL Agent Executor",
        agent=agent,
        tools=tools,
        verbose=True,
        # early_stopping_method="generate",
        handle_parsing_errors=err_handle
)

In [79]:
res = agent.invoke({
    "input": "Select the names of all the people who are in the airline database"
})



> Entering new SQL Agent Executor chain...
Let's get started!

Thought: To answer this question, I need to know what tables and columns exist in the airline database. I'll start by listing all the tables using `sql_db_list_tables`.

Action: sql_db_list_tables
Action Input: (empty stringcompany, pass_in_trip, passenger, tripHere's my response:

Question: Select the names of all the people who are in the airline database
Thought: Let's get started!
Action: sql_db_list_tables
Action Input: (empty stringcompany, pass_in_trip, passenger, tripHere's my response:

Question: Select the names of all the people who are in the airline database
Thought: Let's get started!
Action: sql_db_schema
Action Input: passenger
CREATE TABLE passenger (
	passenger_id BIGSERIAL NOT NULL, 
	passenger_name VARCHAR(60), 
	CONSTRAINT passenger_pkey PRIMARY KEY (passenger_id)
)

/*
3 rows from passenger table:
passenger_id	passenger_name
16	John
17	James
18	Poul
*/Here's the continuation of the thought process:



In [80]:
print(res)

{'input': 'Select the names of all the people who are in the airline database', 'output': 'The names of all the people who are in the airline database are John, James, Poul, Christofer, and Superman.'}


In [81]:
print(res['output'])

The names of all the people who are in the airline database are John, James, Poul, Christofer, and Superman.
